In [1]:
import pandas as pd


In [2]:
weather = pd.read_parquet("quito_weather.parquet");
weather.head()

,ds,station,units,property,item_id,possible_range,possible_units
0,2004-01-01 00:00:00,BELISARIO,42.14,O3,BELISARIO_O3,None,µg/m³
1,2004-01-01 01:00:00,BELISARIO,59.97,O3,BELISARIO_O3,None,µg/m³
2,2004-01-01 02:00:00,BELISARIO,61.73,O3,BELISARIO_O3,None,µg/m³
3,2004-01-01 03:00:00,BELISARIO,71.14,O3,BELISARIO_O3,None,µg/m³
4,2004-01-01 04:00:00,BELISARIO,37.03,O3,BELISARIO_O3,None,µg/m³


In [3]:
# ===============================
# Data Analysis of the raw dataset
# ===============================

print("DATASET SHAPE: ", weather.shape)

print("\nDATA TYPES: ")
display(weather.dtypes)

print("\nMISSING VALUES: ")
display(weather.isnull().sum())

print("\nDUPLICATES: ")
display(weather.duplicated().sum())


DATASET SHAPE:  (20266056, 7)

DATA TYPES: 


ds                datetime64[ns]
station                   object
units                    float64
property                  object
item_id                   object
possible_range            object
possible_units            object
dtype: object


MISSING VALUES: 


ds                       0
station                  0
units              5001222
property                 0
item_id                  0
possible_range    10062354
possible_units           0
dtype: int64


DUPLICATES: 


np.int64(0)

In [4]:

print("\nPROPERTIES: ")
display(weather.property.unique())


PROPERTIES: 


array(['O3', 'PM2.5', 'PRE', 'VEL', 'NO2', 'PM10', 'TMP', 'CO', 'LLU',
       'IUV', 'SO2', 'HUM', 'RS', 'DIR'], dtype=object)

In [5]:
#SPLIT THE PROPERTY COLUMN INTO TMP, HUM AND LLU COLUMNS
groups = weather.groupby('property')
weather_TMP = groups.get_group('TMP')
weather_HUM = groups.get_group('HUM')
weather_LLU = groups.get_group('LLU')

#DROP UNNECESSARY COLUMNS
weather_TMP = weather_TMP.rename(columns={'units':'TMP'})
weather_TMP = weather_TMP.drop(columns=['property', 'item_id', 'possible_range', 'possible_units'])
weather_HUM = weather_HUM.rename(columns={'units':'HUM'})
weather_HUM = weather_HUM.drop(columns=['property', 'item_id', 'possible_range', 'possible_units'])
weather_LLU = weather_LLU.rename(columns={'units':'LLU'})
weather_LLU = weather_LLU.drop(columns=['property', 'item_id', 'possible_range', 'possible_units'])

print("\n WEATHER_TMP_SHAPE")
display(weather_TMP.shape)
print("\n WEATHER_HUM_SHAPE")
display(weather_HUM.shape)
print("\n WEATHER_LLU_SHAPE")
display(weather_HUM.shape)
#COMBINE DATAFRAMES

weather_red = weather_TMP
weather_red = weather_red.merge(weather_HUM, on=['ds', 'station'])
weather_red = weather_red.merge(weather_LLU, on=['ds', 'station'])

display(weather_red.head())
print("\n NEW WEATHER SHAPE")
display(weather_red.shape)







 WEATHER_TMP_SHAPE


(1728747, 3)


 WEATHER_HUM_SHAPE


(1728747, 3)


 WEATHER_LLU_SHAPE


(1728747, 3)

,ds,station,TMP,HUM,LLU
0,2004-01-01 00:00:00,Belisario,9.93,98.06,0.0
1,2004-01-01 01:00:00,Belisario,9.14,98.47,0.1
2,2004-01-01 02:00:00,Belisario,8.71,98.65,0.0
3,2004-01-01 03:00:00,Belisario,8.63,99.03,0.0
4,2004-01-01 04:00:00,Belisario,10.18,86.85,0.0



 NEW WEATHER SHAPE


(1536664, 5)

In [8]:
#Data Analysis of the reduced dataset

display(weather_red.head())

print("\nSTATION VALUES") 
display(weather_red.station.value_counts())

print("\nDATASET SHAPE: ", weather_red.shape)

print("\nDATA TYPES: ")
display(weather_red.dtypes)

print("\nMISSING VALUES: ")
display(weather_red.isnull().sum())

print("\nMISSING VALUES (PERCENTAGE): ")
display((weather_red.isnull().sum()/weather_red.shape[0]) * 100)

print("\nDUPLICATES: ")
display(weather_red.duplicated().sum())

,ds,station,TMP,HUM,LLU
0,2004-01-01 00:00:00,Belisario,9.93,98.06,0.0
1,2004-01-01 01:00:00,Belisario,9.14,98.47,0.1
2,2004-01-01 02:00:00,Belisario,8.71,98.65,0.0
3,2004-01-01 03:00:00,Belisario,8.63,99.03,0.0
4,2004-01-01 04:00:00,Belisario,10.18,86.85,0.0



STATION VALUES


station
Belisario     192083
Carapungo     192083
Cotocollao    192083
ElCamal       192083
LosChillos    192083
Tumbaco       192083
Guamaní       192083
Centro        192083
Name: count, dtype: int64


DATASET SHAPE:  (1536664, 5)

DATA TYPES: 


ds         datetime64[ns]
station            object
TMP               float64
HUM               float64
LLU               float64
dtype: object

...

ds              0
station         0
TMP        308750
HUM        328458
LLU        240743
dtype: int64


MISSING VALUES (PERCENTAGE): 


ds          0.000000
station     0.000000
TMP        20.092226
HUM        21.374744
LLU        15.666600
dtype: float64


DUPLICATES: 


np.int64(0)

In [14]:
# We drop the rows that contain missing data
weather_red_nomissing = weather_red.dropna()

print("\nMISSING VALUES: ")
display(weather_red_nomissing.isnull().sum())

print("\nSHAPE: ")
display(weather_red_nomissing.shape)

print("\n PERCENTAGE OF ROWS REMOVED")
display(100 - (weather_red_nomissing.shape[0]/weather_red.shape[0]) * 100)


MISSING VALUES: 


ds         0
station    0
TMP        0
HUM        0
LLU        0
dtype: int64


SHAPE: 


(1195272, 5)


 PERCENTAGE OF ROWS REMOVED


22.216437685792073

In [13]:

weather_red_nomissing.to_csv("weather_clean.csv", index=False)
